# Inserción de Datos Demográficos

En este notebook cargamos y limpiamos dos fuentes de datos de la World Bank:
- **API_AG.LND.TOTL.K2_DS2_en_excel_v2_85542.xls** (Superficie terrestre en km²)
- **API_SP.POP.TOTL_DS2_en_excel_v2_85347.xls** (Población total)

Los datos se normalizan a formato “largo”, se mapean los países a su código en nuestra tabla `Paises`, y finalmente se insertan en la tabla `Hechos` (anual, `periodo_id=17`).


### 1. Preparación del entorno y conexión

In [1]:
import os
import pandas as pd
import numpy as np
import pymysql
from pymysql.constants import CLIENT
from dotenv import load_dotenv

load_dotenv()
DB_HOST     = os.getenv('DB_HOST')
DB_USER     = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_NAME     = os.getenv('DB_NAME')

conexion = pymysql.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    database=DB_NAME,
    client_flag=CLIENT.MULTI_STATEMENTS
)
cursor = conexion.cursor()


### 2. Carga de los archivos Excel

Leemos ambos ficheros, que tienen 3 filas de metadatos arriba del encabezado.


In [2]:
# Rutas a los archivos
ruta_land  = "../../data/fuentes/demograficos/API_AG.LND.TOTL.K2_DS2_en_excel_v2_85542.xls"
ruta_pop = "../../data/fuentes/demograficos/API_SP.POP.TOTL_DS2_en_excel_v2_85347.xls"

df_land = pd.read_excel(ruta_land, skiprows=3)
df_pop = pd.read_excel(ruta_pop,  skiprows=3)

print("Area:", df_land.shape, "Pop:", df_pop.shape)


Area: (266, 69) Pop: (266, 69)


### 3. Transformación a formato largo

Pivotamos los años (columnas) a filas para ambos dataframes.


In [3]:
# Detectar columnas de año (cuatro dígitos)
year_cols_land  = [c for c in df_land.columns if str(c).isdigit()]
year_cols_pop   = [c for c in df_pop.columns  if str(c).isdigit()]

# Melt superficie terrestre
df_land_long  = df_land.melt(
    id_vars=["Country Name","Country Code","Indicator Name","Indicator Code"],
    value_vars=year_cols_land,
    var_name="Year", value_name="Land_km2"
)
# Melt población total
df_pop_long  = df_pop.melt(
    id_vars=["Country Name","Country Code","Indicator Name","Indicator Code"],
    value_vars=year_cols_pop,
    var_name="Year", value_name="Population"
)

# Convertir Year a int
df_land_long["Year"] = df_land_long["Year"].astype(int)
df_pop_long["Year"]  = df_pop_long["Year"].astype(int)

print(df_land_long.head(2))
print(df_pop_long.head(2))


                  Country Name Country Code      Indicator Name  \
0                        Aruba          ABW  Land area (sq. km)   
1  Africa Eastern and Southern          AFE  Land area (sq. km)   

   Indicator Code  Year  Land_km2  
0  AG.LND.TOTL.K2  1960       NaN  
1  AG.LND.TOTL.K2  1960       NaN  
                  Country Name Country Code     Indicator Name Indicator Code  \
0                        Aruba          ABW  Population, total    SP.POP.TOTL   
1  Africa Eastern and Southern          AFE  Population, total    SP.POP.TOTL   

   Year   Population  
0  1960      54922.0  
1  1960  130072080.0  


### 4. Limpieza de valores

– Reemplazamos valores faltantes (“..”, “-”) y convertimos a `float`.  
– Eliminamos filas sin valor.


In [4]:
for df, col in [(df_land_long, "Land_km2"), (df_pop_long, "Population")]:
    df[col] = (
      df[col]
        .astype(str)
        .replace({"..": "", "-": ""})
        .replace(r'^\s*$', np.nan, regex=True)
    )
    df[col] = pd.to_numeric(df[col], errors="coerce")
    n_missing = df[col].isna().sum()
    print(f"⚠️ {n_missing} filas sin {col} → se descartan")
    df.dropna(subset=[col], inplace=True)


⚠️ 2185 filas sin Land_km2 → se descartan
⚠️ 360 filas sin Population → se descartan


### 5. Mapeo de países (`pais_id`)

– Normalizamos el nombre de país.  
– Aplicamos excepciones para casos especiales.  
– Cargamos `Paises` (nombre_en → código) y mapeamos.


In [5]:
# 5.1 cargar dimensión Paises (codigo, nombre_en)
cursor.execute("SELECT codigo, nombre_en FROM Paises;")
dim_p = { nombre_en.strip().lower(): codigo 
          for codigo, nombre_en in cursor.fetchall() }

# 5.2 excepciones puntuales
exceptions = {
    'netherlands':                    'netherlands (kingdom of the)',
    'turkiye':                        'türkiye',
    'united kingdom':                 'united kingdom of great britain and northern ireland',
    'bahamas, the':                   'bahamas',
    'bolivia':                        'bolivia (plurinational state of)',
    'congo, dem. rep.':               'congo (the democratic republic of the)',
    'congo, rep.':                    'congo',
    "cote d'ivoire":                  "côte d'ivoire",
    'egypt, arab rep.':               'egypt',
    'gambia, the':                    'gambia',
    'hong kong sar, china':           'hong kong',
    'iran, islamic rep.':             'iran (islamic republic of)',
    'korea, rep.':                    'korea (the republic of)',
    'micronesia, fed. states of':     'micronesia (federated states of)',
    'st. vincent and the grenadines': 'saint vincent and the grenadines',
    'tanzania':                       'tanzania, the united republic of',
    'curacao':                        'curaçao',
    "korea, dem. people's rep.":     "korea (the democratic people's republic of)",
    'slovak republic':                'slovakia',
    'venezuela, rb':                  'venezuela (bolivarian republic of)',
    'yemen, rep.':                    'yemen',
    'st. kitts and nevis':            'saint kitts and nevis',
    'st. lucia':                      'saint lucia',
    'macao sar, china':               'macao',
    'lao pdr':                        "lao people's democratic republic",
    'kyrgyz republic':                'kyrgyzstan',
    'russian federation':             'russian federation',
    'moldova':                        'moldova (the republic of)',
    'united states':                  'united states of america',
    'st. martin (french part)':       'saint martin (french part)',
    'british virgin islands':         'virgin islands (british)',
    'micronesia, fed. sts.':     'micronesia (federated states of)',
}

def map_pais(df):
    df["country_norm"] = (
        df["Country Name"]
          .astype(str)
          .str.strip()
          .str.lower()
    )
    df["country_db"] = df["country_norm"].map(lambda x: exceptions.get(x, x))
    df["pais_id"]    = df["country_db"].map(dim_p)
    no = df.loc[df["pais_id"].isna(), "Country Name"].unique()
    if len(no):
        print("⚠️ No mapeados:", no)
    return df[df["pais_id"].notna()]

df_land_long = map_pais(df_land_long)
df_pop_long  = map_pais(df_pop_long)


⚠️ No mapeados: ['Africa Eastern and Southern' 'Africa Western and Central' 'Arab World'
 'Central Europe and the Baltics' 'Channel Islands'
 'Caribbean small states' 'East Asia & Pacific (excluding high income)'
 'Early-demographic dividend' 'East Asia & Pacific'
 'Europe & Central Asia (excluding high income)' 'Europe & Central Asia'
 'Euro area' 'European Union' 'Fragile and conflict affected situations'
 'High income' 'Heavily indebted poor countries (HIPC)' 'IBRD only'
 'IDA & IBRD total' 'IDA total' 'IDA blend' 'IDA only'
 'Latin America & Caribbean (excluding high income)'
 'Latin America & Caribbean'
 'Least developed countries: UN classification' 'Low income'
 'Lower middle income' 'Low & middle income' 'Late-demographic dividend'
 'Middle East & North Africa' 'Middle income'
 'Middle East & North Africa (excluding high income)' 'North America'
 'Namibia' 'OECD members' 'Other small states' 'Pre-demographic dividend'
 'West Bank and Gaza' 'Pacific island small states'
 'Post-d

### 6. Cargar los IDs de los indicadores existentes

Leemos de la tabla `Indicadores` (código → id) para luego mapear nuestros dataframes demográficos.


In [6]:
# 6.1 Obtener (id, codigo) de la tabla Indicadores
cursor.execute("SELECT id, codigo FROM Indicadores;")
dim_ind_map = { codigo: iid for iid, codigo in cursor.fetchall() }

# 6.2 Verificar que todos los códigos que vamos a usar existen en la dimensión
for code in ["AG.LND.TOTL.K2", "SP.POP.TOTL"]:
    if code not in dim_ind_map:
        raise KeyError(f"Falta el indicador `{code}` en la tabla Indicadores.")


### 7. Inserción por lotes en la tabla `Hechos`

Para no bloquear la base de datos, hacemos la inserción en chunks de 1000.


In [7]:
fact_tuples = []

# período anual = 17
PER_ANUAL = 17

# Área terrestre → indicador AG.LND.TOTL.K2
for _, r in df_land_long.iterrows():
    fact_tuples.append((
        r["pais_id"],
        PER_ANUAL,
        int(r["Year"]),
        dim_ind_map["AG.LND.TOTL.K2"],
        float(r["Land_km2"])
    ))

# Población → indicador SP.POP.TOTL
for _, r in df_pop_long.iterrows():
    fact_tuples.append((
        r["pais_id"],
        PER_ANUAL,
        int(r["Year"]),
        dim_ind_map["SP.POP.TOTL"],
        float(r["Population"])
    ))

print("Total filas a insertar en Hechos:", len(fact_tuples))

sql_insert = """
INSERT INTO Hechos
  (pais_id, periodo_id, anio, indicador_id, valor)
VALUES (%s,%s,%s,%s,%s);
"""
batch_size = 1000
total = len(fact_tuples)

for start in range(0, total, batch_size):
    end = min(start + batch_size, total)
    chunk = fact_tuples[start:end]
    cursor.executemany(sql_insert, chunk)
    conexion.commit()
    print(f"✔ Insertadas filas {start+1}–{end}")

cursor.close()
conexion.close()



Total filas a insertar en Hechos: 25577
✔ Insertadas filas 1–1000
✔ Insertadas filas 1001–2000
✔ Insertadas filas 2001–3000
✔ Insertadas filas 3001–4000
✔ Insertadas filas 4001–5000
✔ Insertadas filas 5001–6000
✔ Insertadas filas 6001–7000
✔ Insertadas filas 7001–8000
✔ Insertadas filas 8001–9000
✔ Insertadas filas 9001–10000
✔ Insertadas filas 10001–11000
✔ Insertadas filas 11001–12000
✔ Insertadas filas 12001–13000
✔ Insertadas filas 13001–14000
✔ Insertadas filas 14001–15000
✔ Insertadas filas 15001–16000
✔ Insertadas filas 16001–17000
✔ Insertadas filas 17001–18000
✔ Insertadas filas 18001–19000
✔ Insertadas filas 19001–20000
✔ Insertadas filas 20001–21000
✔ Insertadas filas 21001–22000
✔ Insertadas filas 22001–23000
✔ Insertadas filas 23001–24000
✔ Insertadas filas 24001–25000
✔ Insertadas filas 25001–25577
